In [3]:
# Modelo de algoritmo de correção de erros adaptativo

import os 
import pandas as pd
import pickle
import re
from operator import itemgetter
import random

# Abrir arquivo de dados

a_file = open("data.pkl", "rb")
data = pickle.load(a_file)

# A

modulacao = 'QPSK'
sinalRuido = 6.9
codecRate = 64 # Kbps
codecPayload = 160  # Bytes
pps = 50 # milisegundos
cabecalhoIP = 40 # Bytes 
quantChamadas = 1
bandaDisponivel = 180 # Kbps

availableConfigs = []

for mod, param in data.items():
    if mod == modulacao:
        for snr, config in param.items():
            if snr == sinalRuido:
                for tupla in config:
                    rate = tupla[3]
                    if quantChamadas > 1:
                        bandaRequerida = ((((codecPayload * rate + cabecalhoIP) * 8) * pps) / 1000) * quantChamadas
                        if bandaRequerida <= bandaDisponivel:
                            availableConfigs.append(tupla)
                    else:
                        bandaRequerida = (((codecPayload * rate + cabecalhoIP) * 8) * pps) / 1000
                        if bandaRequerida <= bandaDisponivel:
                            availableConfigs.append(tupla)

tabela = pd.DataFrame(availableConfigs, columns =['MOS', 'BER', 'FEC', 'RATE']) 
print('Configurações possiveis: \n')
print(tabela, '\n')

print('A banda disponível é de:',bandaDisponivel, 'Kbps')
try:
    value = sorted(sorted(availableConfigs, key=lambda element: element[3]), 
                   key=lambda element: element[0], reverse=True)[0]
    print('Melhor configuração: ', value[2])
    print('MOS=', value[0])
except:
    print('Impossivel: Largura de banda insuficiente')
    

Configurações possiveis: 

       MOS       BER           FEC      RATE
0  1.01310  0.013426       No Code  1.000000
1  1.01580  0.013738  RS(255, 247)  1.032389
2  1.01100  0.013359  RS(255, 225)  1.133333
3  0.98320  0.011507  RS(240, 200)  1.200000
4  0.79682  0.008326  RS(255, 205)  1.243902
5  0.65765  0.008112  RS(360, 280)  1.285714
6  1.60310  0.000589  RS(255, 185)  1.378378
7  4.36710  0.000004  RS(255, 165)  1.545455
8  4.50000  0.000000  RS(400, 240)  1.666667 

A banda disponível é de: 180 Kbps
Melhor configuração:  RS(400, 240)
MOS= 4.5


In [2]:
# Fechar arquivo

a_file.close()